In [ ]:
import praw
from collections import defaultdict
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
from matplotlib import font_manager
import numpy as np
from scipy.interpolate import make_interp_spline, BSpline
import os
from enum import Enum

In [ ]:
def load_settings(file_name):
    if not os.path.exists(file_name):
        with open(file_name, 'w') as file:
            file.write('{}')
        raise Exception("The settings.json file was created. Please fill it in with the required data.")

    with open(file_name, 'r') as file:
        settings = json.load(file)
        client_id = settings.get('ClientId')
        client_secret = settings.get('ClientSecret')

    if client_id is None or client_secret is None:
        raise Exception("The settings.json file is missing the required data.")

    return client_id, client_secret

# Usage
file_name = 'settings.json'
try:
    client_id, client_secret = load_settings(file_name)
except Exception as e:
    print(str(e))

In [ ]:
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent="user_agent='reddit:my_subreddit_activity:v1.0 (by u/iamdrnolife)'",
    check_for_async=False
)

In [ ]:
def get_user_activity(username):
    user = reddit.redditor(username)

    subreddit_activity = defaultdict(lambda: {"TotalPosts": 0, "TotalPostsKarma": 0, "TotalComments": 0, "TotalCommentsKarma": 0})

    # Iterate over the user's posts
    print(f"Fetching posts for user: {username}")
    post_count = 0
    for post in user.submissions.new(limit=None):
        subreddit_activity[post.subreddit.display_name]["TotalPosts"] += 1
        subreddit_activity[post.subreddit.display_name]["TotalPostsKarma"] += post.score
        post_count += 1
        if post_count % 100 == 0:
            print(f"Processed {post_count} posts")

    # Iterate over the user's comments
    print(f"Fetching comments for user: {username}")
    comment_count = 0
    for comment in user.comments.new(limit=None):
        subreddit_activity[comment.subreddit.display_name]["TotalComments"] += 1
        subreddit_activity[comment.subreddit.display_name]["TotalCommentsKarma"] += comment.score
        comment_count += 1
        if comment_count % 100 == 0:
            print(f"Processed {comment_count} comments")

    print(f"Finished processing. Total posts: {post_count}, Total comments: {comment_count}")

    # Calculate total activity and total karma for each subreddit
    for subreddit, data in subreddit_activity.items():
        data["TotalActivity"] = data["TotalPosts"] + data["TotalComments"]
        data["TotalKarma"] = data["TotalPostsKarma"] + data["TotalCommentsKarma"]

    return subreddit_activity

def write_to_json(username, data):
    # Sort data by total activity in descending order
    sorted_data = dict(sorted(data.items(), key=lambda item: item[1]['TotalActivity'], reverse=True))
    with open(f"{username}_activity.json", "w") as outfile:
        json.dump(sorted_data, outfile)

def retrieve_stats(username):
    activity = get_user_activity(username)
    write_to_json(username, activity)
    return activity

In [ ]:
# Define colors as hex codes
post_color = '#f1c40f'  # Yellow
comment_color = '#27ae60'  # Green
background_color = '#2c3e50'  
karma_line_color = '#e74c3c'  
karma_text_color = '#ecf0f1'
diagram_padding = 5
y_axis_color = '#ffffff'
circle_color = '#c0392b'
circle_border_color = '#e74c3c'
circle_radius = 1000

# Set font
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 10}

mpl.rc('font', **font)

def plot_top_subreddits(activity, filename, type):

    # Sort data by total activity in descending order
    sorted_data = dict(sorted(activity.items(), key=lambda item: item[1]['TotalActivity'], reverse=True))

    # Get the top 10 subreddits by total activity
    top_subreddits = list(sorted_data.keys())[:10]
    total_posts = [sorted_data[subreddit]["TotalPosts"] for subreddit in top_subreddits]
    total_comments = [sorted_data[subreddit]["TotalComments"] for subreddit in top_subreddits]
    post_karma = [sorted_data[subreddit]["TotalPostsKarma"] for subreddit in top_subreddits]
    comment_karma = [sorted_data[subreddit]["TotalCommentsKarma"] for subreddit in top_subreddits]
    total_karma = [sorted_data[subreddit]["TotalKarma"] for subreddit in top_subreddits]

    # Create a stacked bar chart
    fig, ax1 = plt.subplots(figsize=(34, 16))
    fig.patch.set_facecolor(background_color)
    ax1.set_facecolor(background_color)
    post_bars = ax1.bar(top_subreddits, total_posts, color=post_color, label='Posts')
    comment_bars = ax1.bar(top_subreddits, total_comments, bottom=total_posts, color=comment_color, label='Comments')

    # Add title.
    plt.title(f'{filename} - [{type}]', color=y_axis_color)  # add title to the plot

    # Make x-labels diagonal
    plt.xticks(rotation=45)

    # Add grid lines
    ax1.grid(axis='y', color='grey', linestyle='--', linewidth=0.5)
    ax1.set_ylabel('Total Activity', color=y_axis_color)
    ax1.tick_params(axis='y', labelcolor=y_axis_color)
    ax1.tick_params(axis='x', labelcolor=y_axis_color)
    
    # Add second y-axis
    ax2 = ax1.twinx()
    karma_line = ax2.plot(top_subreddits, total_karma, color=karma_line_color, linewidth=2, label='Karma')
    karma_points = ax2.scatter(top_subreddits, total_karma, color=circle_color, s=circle_radius, edgecolors=circle_border_color, zorder=3)
    ax2.set_ylabel('Total Karma', color=y_axis_color)
    ax2.tick_params(axis='y', labelcolor=y_axis_color)
    
    # Add labels to bars
    for i, v in enumerate(total_posts):
        ax1.text(i, v/2, str(v), color='black', fontweight='bold', ha='center', va='center')
    for i, v in enumerate(total_comments):
        ax1.text(i, total_posts[i] + v/2, str(v), color='black', fontweight='bold', ha='center', va='center')

    # Add total karma to points
    for i, v in enumerate(total_karma):
        ax2.text(i, v, str(v), color=karma_text_color, fontweight='bold', ha='center', va='center', zorder=4)


    fig.tight_layout(pad=diagram_padding)
    
    plt.legend([post_bars, comment_bars, karma_line[0]], ['Posts', 'Comments', 'Karma'], loc="upper right")

    # Create directories if they don't exist
    if not os.path.exists('diagrams'):
        os.makedirs('diagrams')

    # Save the figure before showing it
    plt.savefig(f'diagrams/{filename}.png', bbox_inches='tight')

    plt.show()

In [ ]:
#username_to_get_data_about = 'IAmDrNoLife'

In [ ]:
#user_activity = retrieve_stats(username_to_get_data_about)

In [ ]:
#plot_top_subreddits(user_activity, username_to_get_data_about)

In [ ]:
class UserType(Enum):
    Chad = 1
    Autism = 2
    Boomer = 3
    Me = 4
    Random = 5,
    Normal = 6

In [ ]:
list_of_users = [
    ("I-melted", UserType.Random), 
    ("TheAvatar99", UserType.Chad),
    ("krnnz", UserType.Autism), 
    ("Its_Cmac", UserType.Autism), 
    ("WideEstablishment578", UserType.Boomer), 
    ("IAmdrnolife", UserType.Me),
    ("relxp", UserType.Autism),
    ("bad_apiarist", UserType.Autism),
    ("ricochetblue", UserType.Normal),
    ("zerowo_", UserType.Chad),
]

for user, type in list_of_users:
    user_activity = retrieve_stats(user)
    plot_top_subreddits(user_activity, user, type.name)